# Feature Creation Using Yelp and Zillow Data Set

## Machine Learning Problem

In [134]:
#Import Libraries
import pandas as pd
import numpy as np
import sqlite3
import dateutil.parser as parser
import seaborn as sns
import matplotlib.pyplot as plt
import re
import pickle

## Yelp Data Set extraction: Pittsburgh businesses by ZipCode

In [135]:
#Create Database Connection
DBName = 'yelp_project.db'
conn=sqlite3.connect(DBName)
cursor = conn.cursor()

In [136]:
#Read Business Data and query data from reviews
query = "SELECT b.business_id, r.stars, attributes, date, useful, cool, funny, postal_code FROM review r JOIN \
business b USING (business_id) WHERE CAST(postal_code AS int)>=85000 AND CAST(postal_code AS int)<85010"
business_data = pd.read_sql_query(query,conn)
business_data.head()


,business_id,stars,attributes,date,useful,cool,funny,postal_code
0,YPavuOh2XsnRbLfl0DH2lQ,1,"['Alcohol: full_bar', ""Ambience: {'romantic': ...",2012-12-24,1,0,0,85003
1,YPavuOh2XsnRbLfl0DH2lQ,1,"['Alcohol: full_bar', ""Ambience: {'romantic': ...",2013-03-03,2,0,0,85003
2,YPavuOh2XsnRbLfl0DH2lQ,1,"['Alcohol: full_bar', ""Ambience: {'romantic': ...",2013-03-20,0,0,0,85003
3,YPavuOh2XsnRbLfl0DH2lQ,1,"['Alcohol: full_bar', ""Ambience: {'romantic': ...",2013-08-13,1,0,0,85003
4,YPavuOh2XsnRbLfl0DH2lQ,1,"['Alcohol: full_bar', ""Ambience: {'romantic': ...",2013-09-19,0,0,0,85003


In [170]:
len(business_data)

88510

In [171]:
## Split the column attributes one per row 
attributes_splitted = pd.concat([pd.Series(row['business_id'],row['attributes'].split(',')) for _, row in business_data.dropna().iterrows()]).reset_index()
attributes_splitted.columns = ['Attributes', 'business_id']

In [172]:
#Get Restaurants with price
restprice = attributes_splitted[attributes_splitted.Attributes.str.contains("restaurantspricerange", case=False)].reset_index()
restprice.head()

,index,Attributes,business_id
0,36,'RestaurantsPriceRange2: 2',YPavuOh2XsnRbLfl0DH2lQ
1,78,'RestaurantsPriceRange2: 2',YPavuOh2XsnRbLfl0DH2lQ
2,120,'RestaurantsPriceRange2: 2',YPavuOh2XsnRbLfl0DH2lQ
3,162,'RestaurantsPriceRange2: 2',YPavuOh2XsnRbLfl0DH2lQ
4,204,'RestaurantsPriceRange2: 2',YPavuOh2XsnRbLfl0DH2lQ


In [173]:
#Extract price range and remove non-numerical characters
restprice['Price'] = [ pd.to_numeric(re.sub(r"[\[\]' ]", "", r.split(":")[1])) for r in restprice.Attributes]
len(restprice)

73215

In [141]:
#keep business id and price column
business_price = restprice[['business_id', 'Price']]

In [142]:
#Left Join of both business data and business price data frame
business_data_Price = business_data.join(business_price, how = 'left', rsuffix='_bp')
business_data_Price.head()

,business_id,stars,attributes,date,useful,cool,funny,postal_code,business_id_bp,Price
0,YPavuOh2XsnRbLfl0DH2lQ,1,"['Alcohol: full_bar', ""Ambience: {'romantic': ...",2012-12-24,1,0,0,85003,YPavuOh2XsnRbLfl0DH2lQ,2.0
1,YPavuOh2XsnRbLfl0DH2lQ,1,"['Alcohol: full_bar', ""Ambience: {'romantic': ...",2013-03-03,2,0,0,85003,YPavuOh2XsnRbLfl0DH2lQ,2.0
2,YPavuOh2XsnRbLfl0DH2lQ,1,"['Alcohol: full_bar', ""Ambience: {'romantic': ...",2013-03-20,0,0,0,85003,YPavuOh2XsnRbLfl0DH2lQ,2.0
3,YPavuOh2XsnRbLfl0DH2lQ,1,"['Alcohol: full_bar', ""Ambience: {'romantic': ...",2013-08-13,1,0,0,85003,YPavuOh2XsnRbLfl0DH2lQ,2.0
4,YPavuOh2XsnRbLfl0DH2lQ,1,"['Alcohol: full_bar', ""Ambience: {'romantic': ...",2013-09-19,0,0,0,85003,YPavuOh2XsnRbLfl0DH2lQ,2.0


In [143]:
#Extract year from the date column
years=[pd.to_datetime(row['date']).year for _, row in business_data_Price.iterrows()]

In [144]:
#Veify Length of years
len(years)

88510

In [145]:
#include year column in the complete dataframe
business_data_Price['Years'] = years
business_data_Price

,business_id,stars,attributes,date,useful,cool,funny,postal_code,business_id_bp,Price,Years
0,YPavuOh2XsnRbLfl0DH2lQ,1,"['Alcohol: full_bar', ""Ambience: {'romantic': ...",2012-12-24,1,0,0,85003,YPavuOh2XsnRbLfl0DH2lQ,2.0,2012
1,YPavuOh2XsnRbLfl0DH2lQ,1,"['Alcohol: full_bar', ""Ambience: {'romantic': ...",2013-03-03,2,0,0,85003,YPavuOh2XsnRbLfl0DH2lQ,2.0,2013
2,YPavuOh2XsnRbLfl0DH2lQ,1,"['Alcohol: full_bar', ""Ambience: {'romantic': ...",2013-03-20,0,0,0,85003,YPavuOh2XsnRbLfl0DH2lQ,2.0,2013
3,YPavuOh2XsnRbLfl0DH2lQ,1,"['Alcohol: full_bar', ""Ambience: {'romantic': ...",2013-08-13,1,0,0,85003,YPavuOh2XsnRbLfl0DH2lQ,2.0,2013
4,YPavuOh2XsnRbLfl0DH2lQ,1,"['Alcohol: full_bar', ""Ambience: {'romantic': ...",2013-09-19,0,0,0,85003,YPavuOh2XsnRbLfl0DH2lQ,2.0,2013
5,YPavuOh2XsnRbLfl0DH2lQ,1,"['Alcohol: full_bar', ""Ambience: {'romantic': ...",2013-12-03,0,0,0,85003,YPavuOh2XsnRbLfl0DH2lQ,2.0,2013
6,YPavuOh2XsnRbLfl0DH2lQ,1,"['Alcohol: full_bar', ""Ambience: {'romantic': ...",2014-01-12,5,0,0,85003,YPavuOh2XsnRbLfl0DH2lQ,2.0,2014
7,YPavuOh2XsnRbLfl0DH2lQ,1,"['Alcohol: full_bar', ""Ambience: {'romantic': ...",2014-02-18,0,0,0,85003,YPavuOh2XsnRbLfl0DH2lQ,2.0,2014
8,YPavuOh2XsnRbLfl0DH2lQ,1,"['Alcohol: full_bar', ""Ambience: {'romantic': ...",2014-03-06,0,0,0,85003,YPavuOh2XsnRbLfl0DH2lQ,2.0,2014
9,YPavuOh2XsnRbLfl0DH2lQ,1,"['Alcohol: full_bar', ""Ambience: {'romantic': ...",2014-04-11,2,0,0,85003,YPavuOh2XsnRbLfl0DH2lQ,2.0,2014


In [146]:
#create dummy variables for the number of the stars

zip_matrix = business_data_Price[['postal_code', 'Years', 'stars', 'useful', 'funny', 'cool', 'Price']]
stars = zip_matrix.stars
zip_matrix = pd.get_dummies(zip_matrix, columns = ["stars"])
zip_matrix['stars'] = stars
zip_matrix.head()

,postal_code,Years,useful,funny,cool,Price,stars_1,stars_2,stars_3,stars_4,stars_5,stars
0,85003,2012,1,0,0,2.0,1.0,0.0,0.0,0.0,0.0,1
1,85003,2013,2,0,0,2.0,1.0,0.0,0.0,0.0,0.0,1
2,85003,2013,0,0,0,2.0,1.0,0.0,0.0,0.0,0.0,1
3,85003,2013,1,0,0,2.0,1.0,0.0,0.0,0.0,0.0,1
4,85003,2013,0,0,0,2.0,1.0,0.0,0.0,0.0,0.0,1


Now, we want to aggregate by zipcode and obtain features that describe the distribution

## Obtain mean, median and standard deviation

In [174]:
#aggregate the data by ZIPCODE, calculating the means of each feature
zip_aggregated = zip_matrix.groupby(['postal_code', 'Years']).agg([np.mean, np.std, np.median])
zip_aggregated.head()

useful                      funny                   \
                       mean       std median      mean       std median   
postal_code Years                                                         
85000       2011   2.833333  2.136976    3.0  1.000000  1.095445    1.0   
            2012   1.076923  2.253203    0.0  0.230769  0.832050    0.0   
            2013   3.000000  3.464102    3.0  1.000000  1.414214    0.5   
            2014   0.250000  0.500000    0.0  0.000000  0.000000    0.0   
            2015   0.250000  0.500000    0.0  0.000000  0.000000    0.0   

                       cool                  Price  ...   stars_3 stars_4  \
                       mean       std median  mean  ...    median    mean   
postal_code Years                                   ...                     
85000       2011   0.500000  0.836660    0.0  1.50  ...       0.0     0.0   
            2012   0.461538  0.877058    0.0  1.00  ...       0.0     0.0   
            2013   1.750000  2.872281    0.5  1.25  ...       0.0     0.0   
            2014   0.000000  0.000000    0.0  1.50  ...       0.0     0.0   
            2015   0.000000  0.000000    0.0  2.00  ...       0.0     0.0   

                                stars_5                     stars           \
                   std median      mean      std median      mean      std   
postal_code Years                                                            
85000       2011   0.0    0.0  1.000000  0.00000    1.0  5.000000  0.00000   
            2012   0.0    0.0  0.923077  0.27735    1.0  4.769231  0.83205   
            2013   0.0    0.0  0.750000  0.50000    1.0  4.000000  2.00000   
            2014   0.0    0.0  1.000000  0.00000    1.0  5.000000  0.00000   
            2015   0.0    0.0  1.000000  0.00000    1.0  5.000000  0.00000   

                          
                  median  
postal_code Years         
85000       2011     5.0  
            2012     5.0  
            2013     5.0  
            2014     5.0  
            2015     5.0  

[5 rows x 30 columns]

In [175]:
# change column names to eliminate nested columns
zip_aggregated.columns = ['_'.join(col).strip() for col in zip_aggregated.columns.values]
zip_aggregated.head()

useful_mean  useful_std  useful_median  funny_mean  \
postal_code Years                                                       
85000       2011      2.833333    2.136976            3.0    1.000000   
            2012      1.076923    2.253203            0.0    0.230769   
            2013      3.000000    3.464102            3.0    1.000000   
            2014      0.250000    0.500000            0.0    0.000000   
            2015      0.250000    0.500000            0.0    0.000000   

                   funny_std  funny_median  cool_mean  cool_std  cool_median  \
postal_code Years                                                              
85000       2011    1.095445           1.0   0.500000  0.836660          0.0   
            2012    0.832050           0.0   0.461538  0.877058          0.0   
            2013    1.414214           0.5   1.750000  2.872281          0.5   
            2014    0.000000           0.0   0.000000  0.000000          0.0   
            2015    0.000000           0.0   0.000000  0.000000          0.0   

                   Price_mean      ...       stars_3_median  stars_4_mean  \
postal_code Years                  ...                                      
85000       2011         1.50      ...                  0.0           0.0   
            2012         1.00      ...                  0.0           0.0   
            2013         1.25      ...                  0.0           0.0   
            2014         1.50      ...                  0.0           0.0   
            2015         2.00      ...                  0.0           0.0   

                   stars_4_std  stars_4_median  stars_5_mean  stars_5_std  \
postal_code Years                                                           
85000       2011           0.0             0.0      1.000000      0.00000   
            2012           0.0             0.0      0.923077      0.27735   
            2013           0.0             0.0      0.750000      0.50000   
            2014           0.0             0.0      1.000000      0.00000   
            2015           0.0             0.0      1.000000      0.00000   

                   stars_5_median  stars_mean  stars_std  stars_median  
postal_code Years                                                       
85000       2011              1.0    5.000000    0.00000           5.0  
            2012              1.0    4.769231    0.83205           5.0  
            2013              1.0    4.000000    2.00000           5.0  
            2014              1.0    5.000000    0.00000           5.0  
            2015              1.0    5.000000    0.00000           5.0  

[5 rows x 30 columns]

In [176]:
#drop duplicates to have just one single price per business for each zipcode (the number of reviews do not matter)

business_price_zip = business_data_Price[['business_id','Price', 'postal_code', 'Years']].drop_duplicates()
price_zip = business_price_zip[['postal_code', 'Price', 'Years']]
price_zip.head()

,postal_code,Price,Years
0,85003,2.0,2012
1,85003,2.0,2013
6,85003,2.0,2014
22,85003,2.0,2015
54,85003,2.0,2016


In [177]:
#calculate the price
average_price_zip = price_zip.groupby(['postal_code','Years']).agg([np.mean, np.std, np.median])
average_price_zip.columns = ['_'.join(col).strip() for col in average_price_zip.columns.values]
average_price_zip.head()

Price_mean  Price_std  Price_median
postal_code Years                                     
85000       2011          1.5   0.707107           1.5
            2012          1.0        NaN           1.0
            2013          1.5   0.707107           1.5
            2014          1.5   0.707107           1.5
            2015          2.0        NaN           2.0

In [178]:
#include the price column in the aggregated dataframe
zip_aggregated['Price_mean'] = average_price_zip['Price_mean']
zip_aggregated['Price_median'] = average_price_zip['Price_median']
zip_aggregated['Price_std'] = average_price_zip['Price_std']
zip_aggregated.head()

useful_mean  useful_std  useful_median  funny_mean  \
postal_code Years                                                       
85000       2011      2.833333    2.136976            3.0    1.000000   
            2012      1.076923    2.253203            0.0    0.230769   
            2013      3.000000    3.464102            3.0    1.000000   
            2014      0.250000    0.500000            0.0    0.000000   
            2015      0.250000    0.500000            0.0    0.000000   

                   funny_std  funny_median  cool_mean  cool_std  cool_median  \
postal_code Years                                                              
85000       2011    1.095445           1.0   0.500000  0.836660          0.0   
            2012    0.832050           0.0   0.461538  0.877058          0.0   
            2013    1.414214           0.5   1.750000  2.872281          0.5   
            2014    0.000000           0.0   0.000000  0.000000          0.0   
            2015    0.000000           0.0   0.000000  0.000000          0.0   

                   Price_mean      ...       stars_3_median  stars_4_mean  \
postal_code Years                  ...                                      
85000       2011          1.5      ...                  0.0           0.0   
            2012          1.0      ...                  0.0           0.0   
            2013          1.5      ...                  0.0           0.0   
            2014          1.5      ...                  0.0           0.0   
            2015          2.0      ...                  0.0           0.0   

                   stars_4_std  stars_4_median  stars_5_mean  stars_5_std  \
postal_code Years                                                           
85000       2011           0.0             0.0      1.000000      0.00000   
            2012           0.0             0.0      0.923077      0.27735   
            2013           0.0             0.0      0.750000      0.50000   
            2014           0.0             0.0      1.000000      0.00000   
            2015           0.0             0.0      1.000000      0.00000   

                   stars_5_median  stars_mean  stars_std  stars_median  
postal_code Years                                                       
85000       2011              1.0    5.000000    0.00000           5.0  
            2012              1.0    4.769231    0.83205           5.0  
            2013              1.0    4.000000    2.00000           5.0  
            2014              1.0    5.000000    0.00000           5.0  
            2015              1.0    5.000000    0.00000           5.0  

[5 rows x 30 columns]

## Proportion of reviews per year

Now, we want to calculate the proportion of revies corresponding to each zipcode, from all the reviews in a given year

In [179]:
#Calculations to use in compuyting the proportion of reviews

average_reviews_zip = business_data_Price.groupby(['postal_code','Years']).agg(pd.Series.count)
average_reviews_zip = average_reviews_zip['business_id']
review_byzip = business_data_Price.groupby(['postal_code', 'Years']).count()
total_reviews = business_data_Price.groupby(['Years']).count().business_id
review_byzip = review_byzip.reset_index()[['postal_code','Years','business_id']]


In [180]:
#calculate the proportion of reviews that each zipcode has for each year
prop_reviews = [ float(review_byzip.ix[i]['business_id'])/total_reviews[review_byzip.ix[i]['Years']] for i in range(len(review_byzip))]

In [181]:
#include in aggreged data set
zip_aggregated['reviews_perc'] = prop_reviews
zip_aggregated.head()

useful_mean  useful_std  useful_median  funny_mean  \
postal_code Years                                                       
85000       2011      2.833333    2.136976            3.0    1.000000   
            2012      1.076923    2.253203            0.0    0.230769   
            2013      3.000000    3.464102            3.0    1.000000   
            2014      0.250000    0.500000            0.0    0.000000   
            2015      0.250000    0.500000            0.0    0.000000   

                   funny_std  funny_median  cool_mean  cool_std  cool_median  \
postal_code Years                                                              
85000       2011    1.095445           1.0   0.500000  0.836660          0.0   
            2012    0.832050           0.0   0.461538  0.877058          0.0   
            2013    1.414214           0.5   1.750000  2.872281          0.5   
            2014    0.000000           0.0   0.000000  0.000000          0.0   
            2015    0.000000           0.0   0.000000  0.000000          0.0   

                   Price_mean      ...       stars_4_mean  stars_4_std  \
postal_code Years                  ...                                   
85000       2011          1.5      ...                0.0          0.0   
            2012          1.0      ...                0.0          0.0   
            2013          1.5      ...                0.0          0.0   
            2014          1.5      ...                0.0          0.0   
            2015          2.0      ...                0.0          0.0   

                   stars_4_median  stars_5_mean  stars_5_std  stars_5_median  \
postal_code Years                                                              
85000       2011              0.0      1.000000      0.00000             1.0   
            2012              0.0      0.923077      0.27735             1.0   
            2013              0.0      0.750000      0.50000             1.0   
            2014              0.0      1.000000      0.00000             1.0   
            2015              0.0      1.000000      0.00000             1.0   

                   stars_mean  stars_std  stars_median  reviews_perc  
postal_code Years                                                     
85000       2011     5.000000    0.00000           5.0      0.000910  
            2012     4.769231    0.83205           5.0      0.001762  
            2013     4.000000    2.00000           5.0      0.000392  
            2014     5.000000    0.00000           5.0      0.000286  
            2015     5.000000    0.00000           5.0      0.000223  

[5 rows x 31 columns]

Include changes for each column

In [182]:
flatten_zip = zip_aggregated.reset_index()
for col in flatten_zip.columns[2:]:
    col_name = col + '_change'
    flatten_zip[col_name] = np.where(flatten_zip['postal_code']==flatten_zip['postal_code'].shift(),\
                                     (flatten_zip[col]-flatten_zip[col].shift())/flatten_zip[col].shift(),0)
    #df['u'] = np.where(df['Set']=='Z', 'green', 'red')



In [183]:
flatten_zip.head()

,postal_code,Years,useful_mean,useful_std,useful_median,funny_mean,funny_std,funny_median,cool_mean,cool_std,...,stars_4_mean_change,stars_4_std_change,stars_4_median_change,stars_5_mean_change,stars_5_std_change,stars_5_median_change,stars_mean_change,stars_std_change,stars_median_change,reviews_perc_change
0,85000,2011,2.833333,2.136976,3.0,1.000000,1.095445,1.0,0.500000,0.836660,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000
1,85000,2012,1.076923,2.253203,0.0,0.230769,0.832050,0.0,0.461538,0.877058,...,NaN,NaN,NaN,-0.076923,inf,0.0,-0.046154,inf,0.0,0.936789
2,85000,2013,3.000000,3.464102,3.0,1.000000,1.414214,0.5,1.750000,2.872281,...,NaN,NaN,NaN,-0.187500,0.802776,0.0,-0.161290,1.403701,0.0,-0.777419
3,85000,2014,0.250000,0.500000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,NaN,NaN,NaN,0.333333,-1.000000,0.0,0.250000,-1.000000,0.0,-0.271754
4,85000,2015,0.250000,0.500000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,NaN,NaN,NaN,0.000000,NaN,0.0,0.000000,NaN,0.0,-0.218684


In [184]:
#calculate the previous mean stars that the zipcode received in the previous year
avg_review_prev1 = [flatten_zip[(flatten_zip['postal_code']==flatten_zip.ix[i]['postal_code'])&(flatten_zip['Years']==(flatten_zip.ix[i]['Years']-1))]['stars_mean'].values for i in range(len(flatten_zip))]

In [185]:
#include the previous star mean in the aggregated data frame
zip_aggregated = flatten_zip
zip_aggregated['prev_stars'] = avg_review_prev1
zip_aggregated['prev_stars'] = zip_aggregated['prev_stars'].str.get(0)
zip_aggregated.head()

,postal_code,Years,useful_mean,useful_std,useful_median,funny_mean,funny_std,funny_median,cool_mean,cool_std,...,stars_4_std_change,stars_4_median_change,stars_5_mean_change,stars_5_std_change,stars_5_median_change,stars_mean_change,stars_std_change,stars_median_change,reviews_perc_change,prev_stars
0,85000,2011,2.833333,2.136976,3.0,1.000000,1.095445,1.0,0.500000,0.836660,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,NaN
1,85000,2012,1.076923,2.253203,0.0,0.230769,0.832050,0.0,0.461538,0.877058,...,NaN,NaN,-0.076923,inf,0.0,-0.046154,inf,0.0,0.936789,5.000000
2,85000,2013,3.000000,3.464102,3.0,1.000000,1.414214,0.5,1.750000,2.872281,...,NaN,NaN,-0.187500,0.802776,0.0,-0.161290,1.403701,0.0,-0.777419,4.769231
3,85000,2014,0.250000,0.500000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,NaN,NaN,0.333333,-1.000000,0.0,0.250000,-1.000000,0.0,-0.271754,4.000000
4,85000,2015,0.250000,0.500000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,NaN,NaN,0.000000,NaN,0.0,0.000000,NaN,0.0,-0.218684,5.000000


In [188]:
#save to pickle ------- Checkpoint
import pickle
output_file = open("Feature_Matrix_onlyYelp.dat","wb")
pickle.dump(zip_aggregated, output_file)
output_file.close()

In [189]:
len(zip_aggregated)

104

## Zillow Data Set Extraction and Integration: Median price per square foot


In [118]:
#Integrate Zillow
zip_price = pd.read_csv("Zip_MedianValuePerSqft_AllHomes.csv")

#Select only Pittsburgh
pittsburgh_data_price = zip_price[(zip_price.RegionName>=85000)&(zip_price.RegionName<85010)]

#Select the RegionName and the columns with year
pittsburgh_data_byYear = pd.concat([pittsburgh_data_price.ix[:,1], pittsburgh_data_price[pittsburgh_data_price.columns[7:]]], axis = 1)

#Melt the dataframe to transform the columns into rows
flat_zip = pd.melt(pittsburgh_data_byYear, id_vars="RegionName", var_name="Date", value_name="MedSqft")
flat_zip.head()

,RegionName,Date,MedSqft
0,85008,1996-04,52.0
1,85007,1996-04,61.0
2,85003,1996-04,82.0
3,85004,1996-04,70.0
4,85008,1996-05,53.0


In [119]:
#Extract only the years
years = [flat_zip.Date.ix[i].split('-')[0] for i in range(len(flat_zip))]

#Add to df
flat_zip['Years'] = years

#Select only these columns
flat_zip = flat_zip[['RegionName', 'Years', 'MedSqft']]

#Group by and aggregate by median
med_zip = flat_zip.groupby(['RegionName', 'Years']).median()

#Flatten the table
med_zip = med_zip.reset_index()
med_zip.head()

,RegionName,Years,MedSqft
0,85003,1996,84.0
1,85003,1997,87.0
2,85003,1998,99.5
3,85003,1999,106.5
4,85003,2000,116.0


In [120]:
#match the zip codes in zillow with the zipcodes in YELP by year and obtain the corresponding median price per sqft
Med_price_sqft = [med_zip[(med_zip['RegionName']==int(flatten_zip.ix[i]['postal_code']))&(med_zip['Years']==str(flatten_zip.ix[i]['Years']))]['MedSqft'].values for i in range(len(flatten_zip))]

In [121]:
#include the median price per squarefeet in the aggregated data set
zip_aggregated['MedPrice_sqft'] = Med_price_sqft
zip_aggregated['MedPrice_sqft'] = zip_aggregated['MedPrice_sqft'].str.get(0)
zip_aggregated.head()

useful_mean  useful_std  useful_median  funny_mean  \
postal_code Years                                                       
85000       2011      2.833333    2.136976            3.0    1.000000   
            2012      1.076923    2.253203            0.0    0.230769   
            2013      3.000000    3.464102            3.0    1.000000   
            2014      0.250000    0.500000            0.0    0.000000   
            2015      0.250000    0.500000            0.0    0.000000   

                   funny_std  funny_median  cool_mean  cool_std  cool_median  \
postal_code Years                                                              
85000       2011    1.095445           1.0   0.500000  0.836660          0.0   
            2012    0.832050           0.0   0.461538  0.877058          0.0   
            2013    1.414214           0.5   1.750000  2.872281          0.5   
            2014    0.000000           0.0   0.000000  0.000000          0.0   
            2015    0.000000           0.0   0.000000  0.000000          0.0   

                   Price_mean      ...        stars_4_median  stars_5_mean  \
postal_code Years                  ...                                       
85000       2011          1.5      ...                   0.0      1.000000   
            2012          1.0      ...                   0.0      0.923077   
            2013          1.5      ...                   0.0      0.750000   
            2014          1.5      ...                   0.0      1.000000   
            2015          2.0      ...                   0.0      1.000000   

                   stars_5_std  stars_5_median  stars_mean  stars_std  \
postal_code Years                                                       
85000       2011       0.00000             1.0    5.000000    0.00000   
            2012       0.27735             1.0    4.769231    0.83205   
            2013       0.50000             1.0    4.000000    2.00000   
            2014       0.00000             1.0    5.000000    0.00000   
            2015       0.00000             1.0    5.000000    0.00000   

                   stars_median  reviews_perc  prev_stars  MedPrice_sqft  
postal_code Years                                                         
85000       2011            5.0      0.000910         NaN            NaN  
            2012            5.0      0.001762    5.000000            NaN  
            2013            5.0      0.000392    4.769231            NaN  
            2014            5.0      0.000286    4.000000            NaN  
            2015            5.0      0.000223    5.000000            NaN  

[5 rows x 33 columns]

In [ ]:
#Update Column names

zip_aggregated.columns = [u'useful', u'funny', u'cool', u'Price', u'stars_1', u'stars_2',
       u'stars_3', u'stars_4', u'stars_5', u'stars', u'reviews_perc',
       u'prev_stars', u'MedPrice_sqft(t)']

Now, we wish to include the past median price per square foot (from the zillow data)

In [123]:
#For t=t-1
t=-1
Med_price_sqft_tminus1 = [med_zip[(med_zip['RegionName']==int(flatten_zip.ix[i]['postal_code']))&(med_zip['Years']==str(flatten_zip.ix[i]['Years']+t))]['MedSqft'].values for i in range(len(flatten_zip))]

In [124]:
#For t=t-2
t=-2
Med_price_sqft_tminus2 = [med_zip[(med_zip['RegionName']==int(flatten_zip.ix[i]['postal_code']))&(med_zip['Years']==str(flatten_zip.ix[i]['Years']+t))]['MedSqft'].values for i in range(len(flatten_zip))]

In [125]:
#For t=t+1
t=1
Med_price_sqft_tplus1 = [med_zip[(med_zip['RegionName']==int(flatten_zip.ix[i]['postal_code']))&(med_zip['Years']==str(flatten_zip.ix[i]['Years']+t))]['MedSqft'].values for i in range(len(flatten_zip))]

In [126]:
#Include the previous and T+1 price per square foot in the aggregated data set

zip_aggregated['MedPrice_sqft_tminus1'] = Med_price_sqft_tminus1
zip_aggregated['MedPrice_sqft_tminus1'] = zip_aggregated['MedPrice_sqft_tminus1'].str.get(0)

zip_aggregated['MedPrice_sqft_tminus2'] = Med_price_sqft_tminus2
zip_aggregated['MedPrice_sqft_tminus2'] = zip_aggregated['MedPrice_sqft_tminus2'].str.get(0)

zip_aggregated['MedPrice_sqft_tplus1'] = Med_price_sqft_tplus1
zip_aggregated['MedPrice_sqft_tplus1'] = zip_aggregated['MedPrice_sqft_tplus1'].str.get(0)

#zip_aggregated.head()

In [127]:
#Checkpoint----------------- export data

output_file = open("Feature_Matrix_complete.dat","wb")
pickle.dump(zip_aggregated, output_file)
output_file.close()

#export to CSV
zip_aggregated.to_csv("Feature_Matrix_Yelp_Complete.csv")

In [ ]:
#Import data set

input_file = open("Feature_Matrix_complete.dat","rb")
zip_aggregated = pickle.load(input_file)
input_file.close()
zip_aggregated.head()

Now, we will include the change in price from year to year for each zipcode

In [ ]:
#Growth change 

import math

t=-1
Growth_Y2Y = [math.log(zip_aggregated.ix[i]['MedPrice_sqft_tplus1']/zip_aggregated.ix[i]['MedPrice_sqft(t)']) for i in range(len(zip_aggregated))]

#include in aggregated data set
zip_aggregated['Log_Growth_Y2Y'] = Growth_Y2Y

In [ ]:

#Plot the growth distribution

sns.distplot(zip_aggregated['Log_Growth_Y2Y'].dropna())
plt.show()


#Absolute number of businesses in the corresponding intervals of growth

print(sum(zip_aggregated['Log_Growth_Y2Y']<-0.05))
print(sum((zip_aggregated['Log_Growth_Y2Y']>=-0.05)&(zip_aggregated['Log_Growth_Y2Y']<0)))
print(sum((zip_aggregated['Log_Growth_Y2Y']>=0)&(zip_aggregated['Log_Growth_Y2Y']<0.05)))
print(sum((zip_aggregated['Log_Growth_Y2Y']>=0.05)&(zip_aggregated['Log_Growth_Y2Y']<0.10)))
print(sum((zip_aggregated['Log_Growth_Y2Y']>=0.1)))


In [ ]:
#reset index

zip_aggregated = zip_aggregated.reset_index()
zip_aggregated.head()

# Model Training: Predicting the yearly change in the Medain price per square foot

In [ ]:
#Separate data in train and test sets

train_data = zip_aggregated[zip_aggregated['Years']<2015].dropna()
test_data = zip_aggregated[zip_aggregated['Years']>=2015].dropna()
test_data.head()

In [ ]:
#Assign outcome variable

x_train = train_data[train_data.columns[2:17]]
y_train = train_data['Log_Growth_Y2Y']
x_test = test_data[test_data.columns[2:17]]
y_test = test_data['Log_Growth_Y2Y']

## Linear Regression Model:

In [ ]:
#Import libraries

from sklearn import linear_model
regr = linear_model.LinearRegression()

#Train model
regr.fit(x_train, y_train)

In [ ]:
#The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % np.mean((regr.predict(x_test) - y_test) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x_test, y_test))

In [ ]:
#predict
regr.predict(x_test)

In [ ]:
#Base model: Estimate based on the mean growth and calculate error

mean_growth = np.repeat(y_train.mean(),len(y_test))

np.mean((mean_growth - y_test) ** 2)

In [ ]:

Growth_Y2Y_tminus1 = [(zip_aggregated.ix[i]['MedPrice_sqft(t)']/zip_aggregated.ix[i]['MedPrice_sqft_tminus1'])-1 for i in range(len(zip_aggregated))]
Growth_Y2Y_tminus2 = [(zip_aggregated.ix[i]['MedPrice_sqft_tminus1']/zip_aggregated.ix[i]['MedPrice_sqft_tminus2'])-1 for i in range(len(zip_aggregated))]

In [ ]:
t=-1

stars_Y2Y = [zip_aggregated.ix[i]['stars']/zip_aggregated[((zip_aggregated['postal_code'])==(zip_aggregated.ix[i]['postal_code']))&((zip_aggregated['Years'])==(zip_aggregated.ix[i]['Years']+t))]['stars'].values-1 for i in range(len(zip_aggregated))]

In [ ]:
stars_Y2Y

In [ ]:
zip_aggregated['stars_Y2Y'] = stars_Y2Y
zip_aggregated

In [ ]:
zip_matrix.head()

In [ ]:
#Median
median_values = zip_matrix.groupby(['postal_code', 'Years']).agg(pd.Series.median)

#StdDev
std_values = zip_matrix.groupby(['postal_code', 'Years']).agg(pd.Series.std)

In [ ]:
median_values

In [ ]:
zip_aggregated = zip_aggregated.join(median_values, on = ['postal_code', 'Years'], how = "left", rsuffix="_median")

In [ ]:
zip_aggregated = zip_aggregated.join(std_values, on = ['postal_code', 'Years'], how = "left", rsuffix="_std")

In [ ]:
zip_aggregated

In [ ]:
len(zip_aggregated.dropna())